# Токенизация предложений, получение граммпризнаков и сохранение в json

In [ ]:
import spacy
import json
import csv
from collections import defaultdict

# Загрузим русскую модель spaCy
nlp = spacy.load("ru_core_news_lg")

In [ ]:
def process_sentence(sentence) -> dict:
    """Токенизирует предложение и извлекает информацию о каждом слове."""
    doc = nlp(sentence)
    words_info = {}
    for token in doc:
        words_info[token.text] = {
            "часть речи": token.pos_,
            "лемма": token.lemma_,
            "грамматические признаки": token.morph.to_dict()
        }
    return words_info

def convert_tsv_to_json(input_file) -> dict:
    """Преобразует TSV в требуемый JSON с глобальной нумерацией предложений."""
    data = defaultdict(lambda: defaultdict(dict))
    global_sentence_counter = 0  # Глобальный счетчик для всех предложений

    with open(input_file, "r", encoding="utf-8") as tsvfile:
        reader = csv.DictReader(tsvfile, delimiter="\t")
        for row in reader:
            sentence = row["sentence"]
            name = row["name"]
            link = row["link"]

            # Увеличиваем глобальный счетчик и присваиваем номер предложению
            global_sentence_counter += 1
            sentence_id = f"({global_sentence_counter}) {sentence}"

            # Токенизируем предложение и добавляем в структуру данных
            words_info = process_sentence(sentence)
            data[sentence_id] = {
                "link": link,
                "text": name,
                "words": words_info
            }

    return data

def save_to_json(data, output_file):
    """Сохраняет данные в JSON файл."""
    with open(output_file, "w", encoding="utf-8") as jsonfile:
        json.dump(data, jsonfile, ensure_ascii=False, indent=4)


In [ ]:
# Задаём файлы
input_file = "all_sentences.tsv"
output_file = "all_sentences.json"

# Преобразуем TSV в JSON и сохраняем результат
data = convert_tsv_to_json(input_file)
save_to_json(data, output_file)

print("Готово!")